In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

! pip install transformers==4.27.4

! pip install torchtext==0.10.1

import torch
device = torch.device("cuda")
torch.cuda.init()

from google.colab import drive
drive.mount('/content/gdrive')

from transformers import GPT2Tokenizer, GPT2Config, GPT2LMHeadModel, TextDataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 2.0 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.1
    Uninstalling torchtext-0.15.1:
      Successfully uninstalled torchtext-0.15.1
ERROR: pip's dependency resolver does not currently take into account

In [ ]:
model_path = "/content/gdrive/My Drive/model/chitchat_generator.pt"

tokenizer = GPT2Tokenizer.from_pretrained('gpt2', padding_side='left')
model = GPT2LMHeadModel.from_pretrained('gpt2')

model.load_state_dict(torch.load(model_path, map_location=device))

<All keys matched successfully>

In [ ]:
test_path = '/content/gdrive/My Drive/dataset/combined_test.txt'

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
import re
def getResponse(input_text, model,tokenizer, device):
  input_ids = tokenizer.encode(input_text, return_tensors='pt')
  if(len(input_ids)<=0):
    print(input_text)
    return None
  input_ids = input_ids.to(device)
  model = model.to(device)
  output_ids = model.generate(input_ids,pad_token_id=tokenizer.eos_token_id, max_length=100,early_stopping=True)
  output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
  messages = output_text.split("\n")
  first_bot_response = None
  for message in messages:
    if message.startswith("Bot:"):
        first_bot_response = message.strip()
        break
  return first_bot_response

In [ ]:
print(getResponse("User: Hello, I am feeling very sad",model, tokenizer, device))

Bot: I am sorry to hear that. I am sure you are feeling better.


In [ ]:
with open('/content/gdrive/My Drive/dataset/combined_test.txt', 'r') as f:
    test_data = f.readlines()

In [ ]:
len(test_data)

36273

In [ ]:
def prepare(test_data):
  input = {}
  response = {}
  conv_id = 1
  idx = 1
  for i in range(len(test_data)):
    input[idx]=""
    response[idx]=""
    if(test_data[i]=="\n" or test_data[i]=='\n'):
      idx+=1
  for data in test_data:
    if(conv_id>=idx):
      break
    if(data.startswith("User")):
      input[conv_id]+= data 
    elif(data.startswith("Bot")):
      response[conv_id]+=data
    else:
      conv_id+=1
  return input,response

In [ ]:
input, response = prepare(test_data)

'Bot: Like this one, kind of.\n'

In [ ]:
response2=response.copy()

In [ ]:
from tqdm import tqdm
def getResponses(input, response, model,tokenizer, device):
  generated_response = {}
  with open("/content/gdrive/My Drive/dataset/generated_responses2.json", 'a') as f:
    for idx, text in tqdm(input.items()):
      value = getResponse(text, model,tokenizer, device)
      if(value==None):
        response2[idx]="None"
      generated_response[idx] = value
      f.write(str(idx)+ ":" )
      f.write(str(value))
      f.write("\n")
  return generated_response

In [ ]:
generated_response = getResponses(input, response, model,tokenizer, device)

 11%|█         | 1289/11931 [34:01<3:52:59,  1.31s/it]Input length of input_ids is 104, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 104, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
 39%|███▉      | 4648/11931 [2:00:07<1:24:54,  1.43it/s]Input length of input_ids is 134, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 125, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
 40%|████      | 4794/11931 [2:02:45<1:21:15,  1.46it/s]Input length of input_ids is 109, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 159, but `max_length` is set to 100. Thi

In [ ]:
print(generated_response[1000])

None


In [ ]:
import json

# open file for writing
with open("/content/gdrive/My Drive/dataset/generated_responses2.json", "w") as outfile:
    # write dictionary to file in JSON format
    json.dump(generated_response, outfile)

In [ ]:
! pip install bert-score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 KB 7.4 MB/s eta 0:00:00


In [ ]:
type(response.values())

dict_values

In [ ]:
from bert_score import score
P, R, F1 = score(formatted_response, ref_response, lang="en", verbose=True)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/187 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/109 [00:00<?, ?it/s]

done in 59.09 seconds, 117.51 sentences/sec


In [ ]:
print(P.mean().item())
print(R.mean().item())
print(F1.mean().item())

0.8549818992614746
0.8467355370521545
0.8504859209060669


In [ ]:
from nltk.translate.bleu_score import corpus_bleu
bleu_score = corpus_bleu(formatted_response, list(response.values()))
bleu_score

/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

1.2091368608774406e-231

In [ ]:
type(bleu_score)

float

In [ ]:
! pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rouge import Rouge
rouge = Rouge()
scores = rouge.get_scores(list(generated_response.values()), list(response.values()), avg=False)

In [ ]:
! pip install huggingface_hub


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install bleurt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement bleurt (from versions: none)
ERROR: No matching distribution found for bleurt


In [ ]:
bleurt = load("bleurt", module_type="metric")

NameError: ignored

In [ ]:
from bleurt import score


ModuleNotFoundError: ignored

In [ ]:
reference = "The quick brown fox jumped over the lazy dog"
hypothesis = "The fast brown dog leaped over the lazy fox"

# Initialize Rouge scorer
rouge = Rouge()

# Calculate ROUGE scores
scores = rouge.get_scores(hypothesis, reference)

# Print F1 score
print(scores[0]["rouge-1"]["f"])

0.7777777727777778


In [ ]:
import json

# open file for writing
with open("/content/gdrive/My Drive/dataset/generated_responses.json", "w") as outfile:
    # write dictionary to file in JSON format
    json.dump(generated_response, outfile)

In [ ]:
generated_response[10000]

"That's awesome! I'm sure you'll be able to do"

In [ ]:
with open("/content/gdrive/My Drive/dataset/generated_responses2.json", "r") as rf:
  data = rf.readlines()
len(data)

6944

In [ ]:
formatted_response = []
for res in data:
  res.strip()
  if("Bot" in res):
    res = res[res.find("Bot"):len(res)-1]
  formatted_response.append(res)


In [ ]:
ref_response = list(response.values())[:6944]

In [ ]:
len(formatted_response)

6944

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
bleu_score = corpus_bleu(formatted_response, ref_response)
bleu_score

/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

1.1772661568263073e-231

In [ ]:
from rouge import Rouge
rouge = Rouge()
scores = rouge.get_scores(formatted_response, ref_response, avg=True)

In [ ]:
scores

{'rouge-1': {'r': 0.17164103256486782,
  'p': 0.20402829324257235,
  'f': 0.1560088317554268},
 'rouge-2': {'r': 0.007881734942894323,
  'p': 0.010734824309033305,
  'f': 0.0075306377496917355},
 'rouge-l': {'r': 0.16819465058685804,
  'p': 0.1984153534268628,
  'f': 0.1520916786897518}}